In [ ]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression


from pathlib import Path
from SyringePumpAux import *



In [ ]:
#Path settings
figureTargetFolder = Path(r"C:\Users\bruno.cruz\Downloads\ExperimentalData\Figures")
dataFolder = Path(r"C:\Users\bruno.cruz\Downloads\ExperimentalData\Calibration_timecourse")

#Global Viz settings
sns.set_context('paper') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes
plt.rc("axes.spines", top=False, right=False)

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

SeryngePallete = sns.color_palette("deep")

deviceColors = {"Pump1" : SeryngePallete[0],
                "Pump3" :  SeryngePallete[1],
                "Valve3" : SeryngePallete[2],
                "Valve4" :  SeryngePallete[3]
                }

default_fig_size = (6,4)
default_marker_size = 10.

In [ ]:
ws_path = dataFolder / "calibrations.xlsx" 
d = pd.read_excel(ws_path, sheet_name = 'calibration')
d["DeviceName"] = d["DeviceName"].replace({
    "BlackPump": "Pump1",
    "TransparentPump":"Pump3",
    "ValveBox3": "Valve3",
    "ValveBox4":"Valve4"
    })
d["Date"] = d.apply(lambda row : pd.to_datetime(row["Date"]), axis = 1)
pd.unique(d["DeviceName"])
d

In [ ]:
force_0_intercept = False
for device, device_group in d.groupby("DeviceName"):
    fig = plt.figure(figsize=default_fig_size)
    ax = fig.add_subplot(1,1,1)
    for date, date_group in device_group.groupby(["Date"]):
        xx,yy = date_group["Nprotocol"].values, date_group["water_ul"].values
        regModel = LinearRegression(
            fit_intercept = not(force_0_intercept)).fit(xx.reshape(-1, 1), yy.reshape(-1, 1))
        xx_p = np.array((np.min(xx),np.max(xx))).reshape(-1,1)
        yy_p = regModel.predict(xx_p)

        plt.scatter(xx, yy,
                    color = deviceColors[device], s = default_marker_size**2)

        if not(force_0_intercept):
            plt.plot(xx_p, yy_p, color = deviceColors[device], alpha = 0.3, lw = 1,
            label = str(date) +r", $R^2=$"+
                str(round(regModel.score(xx.reshape(-1, 1), yy.reshape(-1, 1)),4))
                + r", $\beta_1=$" + str(np.round(regModel.coef_[0,0],4))
                + r", $\beta_0=$" + str(np.round(regModel.intercept_[0],4))
                )
        else:
            plt.plot(xx_p, yy_p, color = deviceColors[device], alpha = 0.3, lw = 1,
            label = str(date) +r", $R^2=$"+
                str(round(regModel.score(xx.reshape(-1, 1), yy.reshape(-1, 1)),4))
                + r", $\beta_1=$" + str(np.round(regModel.coef_[0],4))
                )
    if date_group["DeviceType"].iloc[0] == 'Pump':
        plt.xlim((0,13))
    else:
        plt.xlim((20,160))

    plt.title(device)
    plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
    plt.ylim((0,45))
    plt.xlabel("Time/NProtocols")
    plt.ylabel("Average delivered volume")
    saveFigure(fig, rootSaveFolder = figureTargetFolder)
    plt.show()

In [ ]:
#Say I want to deliver 20uL
x_2_predict = 8 #uL
fig = plt.figure(figsize=default_fig_size)
ax = fig.add_subplot(1,1,1)
d = d.sort_values(
   by="Date")

for device, device_group in d.groupby("DeviceName"):
    thisDict = {}
    ii = 0
    for date, date_group in device_group.groupby("Date"):
        xx,yy = date_group["Nprotocol"].values, date_group["water_ul"].values
        regModel = LinearRegression().fit(xx.reshape(-1, 1), yy.reshape(-1, 1))
        if ii == 0:
            xx_p = (x_2_predict - regModel.intercept_[0])/regModel.coef_[0,0]
            xx_0 = xx_p #store the value corresponding to 20uL in the first dya
            thisDict[date] = x_2_predict
        else: #find how much it deliveres on consequent days
            yy_p = regModel.coef_[0,0] * xx_0 + regModel.intercept_[0]
            thisDict[date] = yy_p
        ii += 1
    thisDict = pd.DataFrame(thisDict.items(), columns=['Date', 'outVal'])
    thisDict["Date"] = thisDict["Date"] - thisDict["Date"][0] 
    thisDict["Date_days"] = thisDict.apply(lambda row : row["Date"].days, axis=1)
    plt.plot(thisDict["Date_days"], thisDict["outVal"],
        color = deviceColors[device], marker = 'o', markersize = default_marker_size)
plt.title("Volume delivered" + str(x_2_predict) + "$\mu$L")
plt.xlabel("Days since first calibration")
plt.ylabel("Volume delivered")
saveFigure(fig, rootSaveFolder = figureTargetFolder, title= 'Deviation to deliver 8uL')
plt.show()
thisDict